In [1]:
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np
import requests

In [2]:
actors = pd.read_csv('actors_urls.csv', names=['Actors', 'URLs'])

In [3]:
urls = actors['URLs']

In [24]:
def get_name(soup):
    obj = soup.find('h1')
    if not obj:
        return None
    else: return obj.text.strip()

def get_job(soup):
    obj = soup.find(class_='infobar')
    if not obj:
        return None
    obj = obj.find(class_='itemprop')
    if not obj:
        return None
    else: return obj.text.strip()
    
def get_awards(soup):
    obj = soup.find_all(class_='awards-blurb')
    if not obj:
        return None
    else: return [' '.join(i.text.split()) for i in obj] 

def get_credits(soup):
    obj = soup.find_all(text=re.compile('credits\)'))
    if not obj:
        return None
    creds = 0
    for i in obj:
        try:
            creds += int(re.sub('\D', '', i.strip()))
        except ValueError:
            continue
    obj = soup.find_all(text=re.compile('credit\)'))
    if not obj:
        return creds
    else: creds += len(obj)
    return creds

In [5]:
def get_noms(rows):
    i = rows['Awards']
    if pd.isna(i):
        return None
    i = i.split()
    noms = 0
    try:
        i.index('nominated')
        noms += int(i[i.index('nominated') + 1])
    except ValueError:
        noms += 0
    try:
        i.index('nomination')
        noms += int(i[i.index('nomination') - 1])
    except ValueError:
        noms += 0
    return noms

def get_wins(rows):
    i = rows['Awards']
    if pd.isna(i):
        return None
    i = i.split()
    wins = 0
    try:
        i.index('won')
        wins += int(i[i.index('won') + 1])
    except ValueError:
        wins += 0
    try:
        i.index('win')
        wins += int(i[i.index('win') - 1])
    except ValueError:
        wins += 0
    return wins

In [28]:
actor_list = []
headers = ['Name', 'Job', 'Awards', 'Credits']
for i, url in enumerate(urls):
    if (i + 1) % 20 == 0:
        print(url)
    resp = requests.get(url)
    soup = bs(resp.text, 'lxml')
    name = get_name(soup)
    if name == None:
        continue
    job = get_job(soup)
    awards = get_awards(soup)
    credits = get_credits(soup)
    actor_dict = dict(zip(headers, [name, job, awards, credits]))
    actor_list.append(actor_dict)
    if (i + 1) % 100 == 0:
        actors = pd.DataFrame(actor_list, columns=sorted(headers))
        actors['Awards'] = (actors['Awards'].astype(str)
                            .str.strip('[]').str.replace('\'', '')
                            .str.replace('.', '')
                            .str.lower()
                            .str.replace('for ', '')
                            .str.replace('wins', 'win')
                            .str.replace('nominations', 'nomination'))
        actors['Noms'] = actors.apply(get_noms, axis=1)
        actors['Wins'] = actors.apply(get_wins, axis=1)
        actors.to_csv('actors.csv', mode='a', index=False, header=False)
        actors_list = []

https://www.imdb.com/name/nm0000354/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0000245/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0000658/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0000146/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0159507/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0000174/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0000474/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0000201/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0000870/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0001691/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0000620/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0001505/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0000652/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0941777/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0007107/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0001041/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0451234/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0000610/?ref_=fn_al_nm_1
https://www.imdb.com/name/nm0352032/?ref_=fn_a

In [19]:
a = 'https://www.imdb.com/name/nm0038355/?ref_=fn_al_nm_1'
resp = requests.get(a)
soup = bs(resp.text, 'lxml')

In [34]:
len(actor_list)

4774

In [35]:
actors = pd.DataFrame(actor_list, columns=sorted(headers))
actors['Awards'] = (actors['Awards'].astype(str)
                    .str.strip('[]').str.replace('\'', '')
                    .str.replace('.', '')
                    .str.lower()
                    .str.replace('for ', '')
                    .str.replace('wins', 'win')
                    .str.replace('nominations', 'nomination'))
actors['Noms'] = actors.apply(get_noms, axis=1)
actors['Wins'] = actors.apply(get_wins, axis=1)
actors.to_csv('actors_info.csv', mode='a', index=False, header=['Awards', 'Credits', 'Job', 'Name', 'Noms', 'Wins'])